# LangChain Experiement Number 1

In [32]:
import openai
import os
import dotenv
dotenv.load_dotenv()
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

## Sample Project

In [33]:
project_details="""
Basically you would put your hackathon idea in a text box
and we would run like 3 llm chains in the background
One for frontend one for backend, and then feed both chains into the last one which evaluates the two first chains if its feasible or not, and if it isn't, calls the first two chains again
The first two will also return design specifications, eg an outline of the frontend and backend
maybe another textbox in the frontend for skills/target categories
"""
project_technologies="""
OpenAI API
StreamLit
"""

### Setting Up Chain No. 1: Feature Retrieval

In [34]:

llm = ChatOpenAI(temperature=0, max_tokens=1000, model="gpt-3.5-turbo")
prompt = PromptTemplate(
    input_variables=["project_details", "project_technologies"],
    template="""
    Given the following project description and tech stack, identify and elaborate on the key frontend features that would be necessary for development. The frontend features should not involve backend api calls or database interactions. The frontend features should be described in terms of user stories or detailed feature requirements.
    This project is a hackathon project. Break apart the features into MVP and additional features. The MVP should be the minimum features necessary to have a working prototype.
    Project description: {project_details}
    Technologies: {project_technologies}
    """
)

chain = LLMChain(llm=llm, prompt=prompt)
frontend_features = chain.run({
    'project_details': project_details,
    'project_technologies': project_technologies
})
print(frontend_features)

MVP Frontend Features:
1. User Input: The frontend should have a text box where users can input their hackathon idea.
2. Display Design Specifications: The frontend should display the design specifications generated by the first chain, which outlines the frontend and backend components.
3. Feasibility Evaluation: The frontend should display the evaluation result from the last chain, indicating whether the idea is feasible or not.
4. Retry Functionality: If the idea is deemed not feasible, the frontend should provide a button or option to retry the process, triggering the first two chains again.

Additional Frontend Features:
1. Skills/Target Categories Input: The frontend can include an additional text box where users can input their skills or target categories related to their hackathon idea. This information can be used to enhance the evaluation process.
2. Real-time Feedback: The frontend can provide real-time feedback to users as they input their hackathon idea, such as suggestions

## Setting Up Chain Two:  Feature Extraction

In [35]:
specification_prompt = PromptTemplate(
    input_variables=["frontend_features", "project_technologies"],
    template="""
    Given the extracted frontend features {frontend_features} and the specified skills/technologies {project_technologies}, create a detailed technical specification. 
    This specification should include the technologies to be used, the architecture, pages to be developed, and the components required for each page.
    You should ignore the technologies for the backend and focus on the frontend.
    Please also mention any other technical considerations.
    """
)

specification_chain = LLMChain(llm=llm, prompt=specification_prompt)
specification = specification_chain.run({
    'frontend_features': frontend_features,
    'project_technologies': project_technologies
})
print(specification)

Technical Specification:

Technologies:
1. OpenAI API: This API will be used to generate design specifications and evaluate the feasibility of the hackathon idea.
2. Streamlit: This framework will be used to develop the frontend application.

Architecture:
The frontend application will follow a client-server architecture, with the Streamlit application running on the client-side and interacting with the OpenAI API on the server-side.

Pages:
1. Home Page:
   - User Input: This page will include a text box where users can input their hackathon idea.
   - Skills/Target Categories Input (Additional Feature): This page will also include an additional text box where users can input their skills or target categories related to their hackathon idea.

2. Design Specifications Page:
   - Display Design Specifications: This page will display the design specifications generated by the first chain, outlining the frontend and backend components.

3. Feasibility Evaluation Page:
   - Feasibility Eva

## Setting Up Chain Three: AI Approval

In [36]:
approval_prompt = PromptTemplate(
    input_variables=["technical_specification", "aspect", "group_size"],
    template="""
    Given the developed technical specification {technical_specification}, conduct a thorough review for any inconsistencies or issues. 
    This specification is only for the {aspect} aspect of the project, and should not be evaluated for other aspects, such as backend and database requirements.
    Also, evaluate whether the project, as described, can be realistically completed within the two day hackathon for {group_size} people.
    If it meets all the criteria, output '1'. If not, output '0'. In the next line, provide comments explaining the reasons for disapproval, if any.
    """
)

approval_chain = LLMChain(llm=llm, prompt=approval_prompt)
approval = approval_chain.run({
    'technical_specification': specification,
    'aspect': 'frontend',
    'group_size': '4'
})
print(approval)

0
The project, as described, cannot be realistically completed within the two-day hackathon for 4 people. The technical specification includes additional features such as Skills/Target Categories Input, Progress Indicators, Error Handling, User-Friendly Interface, Mobile Responsiveness, which may require additional time and resources to implement. Additionally, integrating with the OpenAI API and ensuring data validation, security, performance optimization, testing, and documentation will also require significant effort. It is recommended to prioritize and simplify the scope of the project to make it feasible within the given constraints.
